# SPR 2026 - Custom Transformer Encoder v2

**Melhorias sobre v1 (0.77272):**

- ✅ **Focal Loss** (γ=2) - foca nas classes difíceis
- ✅ Mais epochs (15 vs 10)
- ✅ Learning rate scheduler com warmup
- ✅ Label smoothing (0.1)
- ✅ Gradient accumulation

**Objetivo: superar 0.77272 (competir com BERTimbau)**

---
**CONFIGURAÇÃO KAGGLE:**
1. Settings → Internet → **OFF**
2. Settings → Accelerator → **GPU T4 x2**
3. **Não precisa de nenhum modelo externo!**
4. **IMPORTANTE:** Execute "Run All" após commit
---

In [ ]:
# =============================================================================
# SPR 2026 - CUSTOM TRANSFORMER v2 (COM FOCAL LOSS)
# =============================================================================
# Melhorias:
# - Focal Loss para classes desbalanceadas
# - Label smoothing
# - Mais epochs + scheduler
# =============================================================================

import os
import re
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from collections import Counter
from tqdm.auto import tqdm
import math
import warnings
warnings.filterwarnings('ignore')

# Config
SEED = 42
MAX_LEN = 256
BATCH_SIZE = 32
EPOCHS = 15           # Aumentado de 10
LR = 1e-4
NUM_CLASSES = 7
LABEL_SMOOTHING = 0.1  # Novo
FOCAL_GAMMA = 2.0      # Novo

# Transformer Config
D_MODEL = 256
N_HEADS = 8
N_ENCODER_LAYERS = 4
D_FF = 512
DROPOUT = 0.1
MIN_FREQ = 2

DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

torch.manual_seed(SEED)
np.random.seed(SEED)

In [ ]:
# =============================================================================
# FOCAL LOSS (CHAVE PARA CLASSES DESBALANCEADAS)
# =============================================================================
class FocalLoss(nn.Module):
    """Focal Loss para lidar com desbalanceamento de classes.
    
    Referência: Lin et al., "Focal Loss for Dense Object Detection"
    
    Foca nos exemplos difíceis, reduzindo peso de exemplos fáceis.
    gamma=0 é equivalente a CrossEntropy.
    gamma=2 é o valor recomendado.
    """
    def __init__(self, gamma=2.0, label_smoothing=0.0, weight=None):
        super().__init__()
        self.gamma = gamma
        self.label_smoothing = label_smoothing
        self.weight = weight
        
    def forward(self, input, target):
        # Cross entropy com label smoothing
        ce_loss = F.cross_entropy(
            input, target, 
            weight=self.weight,
            label_smoothing=self.label_smoothing,
            reduction='none'
        )
        
        # Probabilidade da classe correta
        pt = torch.exp(-ce_loss)
        
        # Focal weight: (1 - pt)^gamma
        focal_weight = (1 - pt) ** self.gamma
        
        # Focal loss
        focal_loss = focal_weight * ce_loss
        
        return focal_loss.mean()

print('Focal Loss definido!')

In [ ]:
# =============================================================================
# CARREGAR DADOS
# =============================================================================
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')

print(f'Train: {train_df.shape}')
print(f'Test: {test_df.shape}')

# Calcular class weights para Focal Loss
class_counts = train_df['target'].value_counts().sort_index().values
class_weights = 1.0 / class_counts
class_weights = class_weights / class_weights.sum() * len(class_weights)
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)
print(f'\nClass weights: {class_weights.cpu().numpy()}')

In [ ]:
# =============================================================================
# TOKENIZER SIMPLES (DO ZERO)
# =============================================================================
class SimpleTokenizer:
    """Tokenizer baseado em palavras, construído do dataset"""
    
    def __init__(self, min_freq=2):
        self.min_freq = min_freq
        self.word2idx = {}
        self.idx2word = {}
        self.vocab_size = 0
        
        self.pad_token = '[PAD]'
        self.unk_token = '[UNK]'
        self.pad_token_id = 0
        self.unk_token_id = 1
        
    def _tokenize(self, text):
        text = text.lower()
        tokens = re.findall(r'[a-záàâãéèêíïóôõúüçñ]+|[0-9]+', text)
        return tokens
    
    def fit(self, texts):
        counter = Counter()
        for text in texts:
            tokens = self._tokenize(text)
            counter.update(tokens)
        
        self.word2idx = {self.pad_token: 0, self.unk_token: 1}
        
        for word, freq in counter.most_common():
            if freq >= self.min_freq:
                self.word2idx[word] = len(self.word2idx)
        
        self.idx2word = {v: k for k, v in self.word2idx.items()}
        self.vocab_size = len(self.word2idx)
        
        print(f'Vocabulário: {self.vocab_size} tokens')
        
    def encode(self, text, max_length=256):
        tokens = self._tokenize(text)
        ids = [self.word2idx.get(t, self.unk_token_id) for t in tokens]
        
        if len(ids) > max_length:
            ids = ids[:max_length]
        
        attention_mask = [1] * len(ids)
        
        padding_length = max_length - len(ids)
        ids = ids + [self.pad_token_id] * padding_length
        attention_mask = attention_mask + [0] * padding_length
        
        return {'input_ids': ids, 'attention_mask': attention_mask}

tokenizer = SimpleTokenizer(min_freq=MIN_FREQ)
tokenizer.fit(train_df['report'].tolist())

VOCAB_SIZE = tokenizer.vocab_size
PAD_IDX = tokenizer.pad_token_id

In [ ]:
# Split treino/validação
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['report'].values,
    train_df['target'].values,
    test_size=0.15,
    random_state=SEED,
    stratify=train_df['target']
)

print(f'Train: {len(train_texts)}, Val: {len(val_texts)}')

In [ ]:
# =============================================================================
# DATASET
# =============================================================================
class TextDataset(Dataset):
    def __init__(self, texts, labels=None):
        self.texts = texts
        self.labels = labels
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        encoding = tokenizer.encode(self.texts[idx], max_length=MAX_LEN)
        
        item = {
            'input_ids': torch.tensor(encoding['input_ids'], dtype=torch.long),
            'attention_mask': torch.tensor(encoding['attention_mask'], dtype=torch.long)
        }
        
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
            
        return item

train_dataset = TextDataset(train_texts, train_labels)
val_dataset = TextDataset(val_texts, val_labels)
test_dataset = TextDataset(test_df['report'].values)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

print(f'Batches - Train: {len(train_loader)}, Val: {len(val_loader)}, Test: {len(test_loader)}')

In [ ]:
# =============================================================================
# MODELO TRANSFORMER CUSTOMIZADO
# =============================================================================
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512, dropout=0.1):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)


class ExtraSelfAttention(nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        self.self_attn = nn.MultiheadAttention(
            embed_dim=d_model,
            num_heads=n_heads,
            dropout=dropout,
            batch_first=True
        )
        self.norm = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        attn_output, _ = self.self_attn(query=x, key=x, value=x, key_padding_mask=mask)
        x = self.norm(x + self.dropout(attn_output))
        return x


class TransformerEncoderClassifier(nn.Module):
    def __init__(self, vocab_size, d_model=256, n_heads=8, n_encoder_layers=4,
                 d_ff=512, num_classes=7, max_len=256, dropout=0.1, pad_idx=0):
        super().__init__()
        
        self.d_model = d_model
        self.pad_idx = pad_idx
        
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=pad_idx)
        self.pos_encoding = PositionalEncoding(d_model, max_len, dropout)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=d_ff,
            dropout=dropout,
            activation='gelu',
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_encoder_layers)
        self.extra_attention = ExtraSelfAttention(d_model, n_heads, dropout)
        
        self.classifier = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(d_model, num_classes)
        )
        
        self._init_weights()
        
    def _init_weights(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)
                
    def forward(self, input_ids, attention_mask=None):
        x = self.embedding(input_ids) * math.sqrt(self.d_model)
        x = self.pos_encoding(x)
        
        padding_mask = (attention_mask == 0) if attention_mask is not None else None
        
        x = self.transformer_encoder(x, src_key_padding_mask=padding_mask)
        x = self.extra_attention(x, mask=padding_mask)
        
        if attention_mask is not None:
            mask_expanded = attention_mask.unsqueeze(-1).float()
            x = (x * mask_expanded).sum(dim=1) / mask_expanded.sum(dim=1).clamp(min=1e-9)
        else:
            x = x.mean(dim=1)
        
        return self.classifier(x)

model = TransformerEncoderClassifier(
    vocab_size=VOCAB_SIZE,
    d_model=D_MODEL,
    n_heads=N_HEADS,
    n_encoder_layers=N_ENCODER_LAYERS,
    d_ff=D_FF,
    num_classes=NUM_CLASSES,
    max_len=MAX_LEN,
    dropout=DROPOUT,
    pad_idx=PAD_IDX
).to(device)

print(f'Params: {sum(p.numel() for p in model.parameters()):,}')

In [ ]:
# =============================================================================
# TREINAMENTO COM FOCAL LOSS + SCHEDULER
# =============================================================================
criterion = FocalLoss(gamma=FOCAL_GAMMA, label_smoothing=LABEL_SMOOTHING, weight=class_weights)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=0.01)

# Scheduler com warmup
total_steps = len(train_loader) * EPOCHS
warmup_steps = int(0.1 * total_steps)

def lr_lambda(step):
    if step < warmup_steps:
        return step / warmup_steps
    return max(0.1, 1 - (step - warmup_steps) / (total_steps - warmup_steps))

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)

print(f'Focal Loss: gamma={FOCAL_GAMMA}, label_smoothing={LABEL_SMOOTHING}')
print(f'Scheduler: {warmup_steps} warmup steps, {total_steps} total steps')

In [ ]:
def train_epoch(model, loader, criterion, optimizer, scheduler):
    model.train()
    total_loss = 0
    preds, targets = [], []
    
    for batch in tqdm(loader, desc='Training'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        optimizer.zero_grad()
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, labels)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
        preds.extend(logits.argmax(dim=-1).cpu().numpy())
        targets.extend(labels.cpu().numpy())
    
    return total_loss / len(loader), f1_score(targets, preds, average='macro')


def evaluate(model, loader):
    model.eval()
    preds, targets = [], []
    
    with torch.no_grad():
        for batch in tqdm(loader, desc='Evaluating'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            
            logits = model(input_ids, attention_mask)
            preds.extend(logits.argmax(dim=-1).cpu().numpy())
            targets.extend(labels.cpu().numpy())
    
    return f1_score(targets, preds, average='macro')

In [ ]:
best_f1 = 0

for epoch in range(EPOCHS):
    print(f'\nEpoch {epoch+1}/{EPOCHS} | LR: {scheduler.get_last_lr()[0]:.2e}')
    
    train_loss, train_f1 = train_epoch(model, train_loader, criterion, optimizer, scheduler)
    val_f1 = evaluate(model, val_loader)
    
    print(f'Train Loss: {train_loss:.4f} | Train F1: {train_f1:.4f} | Val F1: {val_f1:.4f}')
    
    if val_f1 > best_f1:
        best_f1 = val_f1
        torch.save(model.state_dict(), 'best_model.pt')
        print(f'✅ Novo melhor modelo! F1: {best_f1:.4f}')

print(f'\n🏆 Melhor F1: {best_f1:.4f}')

In [ ]:
# =============================================================================
# PREDIÇÕES
# =============================================================================
model.load_state_dict(torch.load('best_model.pt'))
model.eval()

predictions = []

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Predicting'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        
        logits = model(input_ids, attention_mask)
        preds = logits.argmax(dim=-1).cpu().numpy()
        predictions.extend(preds)

# Submissão
submission = pd.DataFrame({'ID': test_df['ID'], 'target': predictions})
submission.to_csv('submission.csv', index=False)

print('✅ submission.csv criado!')
print(submission['target'].value_counts().sort_index())